## Aim

I want to improve the script I did yesterday. Specifically, I want to scrape the session info of a paper and correct mistakes. 

In [1]:
import pandas as pd
import numpy as np
import time 
import math
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select

In [2]:
def click_on_view_program():
    all_btn = driver.find_elements(
        By.CSS_SELECTOR, 
        "div.menu_item__icon_text_window__text > a.mainmenu_text"
    )
    for btn in all_btn:
        if 'Program' in btn.text:
            view_program_btn = btn 
            break
    view_program_btn.click()

In [3]:
def click_on_individual_presentations():
    '''
    To click on 'individual presentations'
    '''
    presentations = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        '//td[@class="tab_topped_window__tab_cell"][2]'
    )))
    presentations.click()

In [4]:
def get_papers():
    """
    get all paper elements in the current page
    """
    papers = driver.find_elements(
        By.CSS_SELECTOR, 'tr.worksheet_window__row__light, tr.worksheet_window__row__dark'
    )
    return papers

def get_paper_meta(paper, year, paper_meta_dict_list):
    """
    get paper index, paper title, and paper_type
        the author names can be found here but I'll collect later in the view page
    """
    idx = paper.find_element(
        By.CSS_SELECTOR, 'td[title="##"]').text
    ## in the format of '0001'
    paper_id = year + '-' + idx.zfill(4)
    # summary elements:
    summary = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Summary"]'
    )
    title = summary.find_element(
        By.CSS_SELECTOR, 'a.search_headingtext'
    ).text
    summary_info = summary.find_elements(
        By.CSS_SELECTOR, 'td[style="padding: 5px;"] tr'
    )
    session = np.nan
    session_submission_division = np.nan
    individual_submission_division = np.nan
    session_submission_subdivision = np.nan
    submission_type = session_submission_subdivision
    for i in summary_info:
        if 'In Session Submission' in i.text:
            session = i.text.lstrip('  In Session Submission: ')
        elif 'Session Submission Division' in i.text:
            session_submission_division = i.text.lstrip('  Session Submission Division: ')
        elif 'Individual Submission Division' in i.text:
            individual_submission_division = i.text.lstrip('  Individual Submission Division: ')
        elif 'Session Submission Sub-Division' in i.text:
            session_submission_subdivision = i.text.lstrip('  Session Submission Sub-Division: ')
        elif 'Submission type' in i.text:
            submission_type = i.text.lstrip('  Individual Submission type: ')
    paper_meta_dict = {
        'Paper ID': paper_id,
        'Title': title,
        'Session': session,
        'Session Division': session_submission_division,
        'Individual Division': individual_submission_division,
        'Session SubDivision': session_submission_subdivision,
        'Sumission Type': submission_type,
    }
    # update the dict list
    paper_meta_dict_list.append(paper_meta_dict)
    return paper_meta_dict

def open_view(paper):
    """
    Input:
        paper element
    Aim:
        open a new window and click 'view'
    """
    action = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Action"]'
    )
    view_link_e = action.find_element(
                By.CSS_SELECTOR, "li.action_list > a.fieldtext"
            )
    view_link = view_link_e.get_attribute('href')
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(view_link)

def get_title_to_check(paper_meta_dict_list):
    # there are two 'tr.header font.headingtext'
    # title is the second one
    headingtexts = driver.find_elements(
        By.CSS_SELECTOR, 'tr.header font.headingtext'
    )
    title_to_check = headingtexts[1].text
    # update the most recent paper_meta_dict_list
    paper_meta_dict_list[-1]['Title to Check'] = title_to_check
    return title_to_check

def get_session_to_check(paper_meta_dict_list):
    session_to_check = driver.find_element(
        By.CSS_SELECTOR, 'blockquote.tight > a.search_headingtext'
    )
    session_to_check = session_to_check.text
    # update the most recent paper_meta_dict_list
    paper_meta_dict_list[-1]['Session to Check'] = session_to_check
    return session_to_check

def get_authors(paper_meta_dict, author_dict_list):
    paper_id, title = paper_meta_dict['Paper ID'], paper_meta_dict['Title']
    # note that authors_e will return a list since there might be multiple authors
    authors = driver.find_elements(
        By.CSS_SELECTOR, 'a.search_fieldtext_name'
    )
    for author in authors:
        author_idx = authors.index(author) + 1
        authorNum = len(authors)
        author_elements = author.text.split(' (')
        author_name = author_elements[0]
        # doc: https://docs.python.org/3.4/library/stdtypes.html?highlight=strip#str.rstrip
        # some don't contain '()', i.e., affiliation info
        try:
            author_aff = author_elements[1].rstrip(')')
        except:
            author_aff = np.nan
        author_dict = {
            'Paper ID': paper_id,
            'Paper Title': title,
            'Number of Authors': authorNum,
            'Author Position': author_idx,
            'Author Name': author_name,
            'Author Affiliation': author_aff,
        }
        author_dict_list.append(author_dict)

def get_abstract(paper_meta_dict_list):
    # abstract
    abstract = driver.find_elements(
        By.CSS_SELECTOR, 'blockquote.tight'
    )[-1]
    abstract = abstract.text
    paper_meta_dict_list[-1]['Abstract'] = abstract
    return abstract

def scrape_one_page(year, page_num):
    papers = get_papers()
    for paper in papers:
        paper_idx = papers.index(paper) + 1
        paper_meta_dict = get_paper_meta(paper, year, paper_meta_dict_list)
        open_view(paper)
        get_title_to_check(paper_meta_dict_list)
        get_session_to_check(paper_meta_dict_list)
        get_authors(paper_meta_dict, author_dict_list)
        get_abstract(paper_meta_dict_list)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        print(f'Page {page_num} Paper {paper_idx} is done')
        time.sleep(0.5)

In [5]:
# # nth-child(1) means there are two div.iterators, I only need one of the two
def get_iterators():
    iterators = driver.find_elements(
        By.CSS_SELECTOR, "div.iterator:nth-child(1) > form a.fieldtext"
    )
    return iterators

In [96]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [97]:
url = 'https://convention2.allacademic.com/one/ica/ica09'
driver.get(url)

In [98]:
click_on_view_program()
click_on_individual_presentations()

## Update the paper meta

In [99]:
papers = get_papers()
paper = papers[0]

In [100]:
idx = paper.find_element(
    By.CSS_SELECTOR, 'td[title="##"]').text
idx

'1'

In [101]:
summary = paper.find_element(
        By.CSS_SELECTOR, 'td[title="Summary"]'
    )
title = summary.find_element(
    By.CSS_SELECTOR, 'a.search_headingtext'
).text
title

'100 World Trade Center Bombings in Darfur Sudan: Ownership Structure and Functions of the Press in a Globalizing World'

In [85]:
def removeprefix(text, prefix):
    # https://stackoverflow.com/a/16891418
    if text.startswith(prefix):
        return text[len(prefix):]
    return text 

In [88]:
summary_info = summary.find_elements(
        By.CSS_SELECTOR, 'td[style="padding: 5px;"] tr'
    )
session = np.nan
session_submission_division = np.nan
submission_type = np.nan
for i in summary_info:
    if 'In Session Submission' in i.text:
        session = removeprefix(i.text, '  In Session Submission: ')
    elif 'Session Submission Division' in i.text:
        session_submission_division = removeprefix(i.text, '  Session Submission Division: ')
    elif 'Session Submission Unit' in i.text:
        session_submission_division = removeprefix(i.text, '  Session Submission Unit: ')
    elif 'Submission type' in i.text:
        submission_type = removeprefix(i.text, '  Individual Submission type: ')

In [89]:
# summary_info = summary.find_elements(
#     By.CSS_SELECTOR, 'td[style="padding: 5px;"] tr'
# )
# for i in summary_info:
#     if 'In Session Submission' in i.text:
#         session = i.text.lstrip('  In Session Submission: ')
#     elif 'Session Submission Division' in i.text:
#         session_submission_division = i.text.lstrip('  Session Submission Division: ')
#     elif 'Individual Submission Division' in i.text:
#         individual_submission_division = i.text.lstrip('  Individual Submission Division: ')
#     elif 'Session Submission Sub-Division' in i.text:
#         session_submission_subdivision = i.text.lstrip('  Session Submission Sub-Division: ')
#     elif 'Submission type' in i.text:
#         submission_type = i.text.lstrip('  Individual Submission type: ')

In [90]:
session

'China\x92s Olympic Makeover: Shaping China\x92s Global Image via the 2008 Beijing Olympics'

In [91]:
session_submission_division

'Intercultural & Development Communication'

In [92]:
submission_type

'Session Paper'

## Update author

In [193]:
# open_view(paper)

In [194]:
# paper_meta_dict_list = []
# author_dict_list = []

In [195]:
# headingtexts = driver.find_elements(
#     By.CSS_SELECTOR, 'tr.header font.headingtext'
# )
# title_to_check = headingtexts[1].text
# title_to_check

In [196]:
# session_to_check = driver.find_element(
#     By.CSS_SELECTOR, 'blockquote.tight > a.search_headingtext'
# )
# session_to_check.text

In [197]:
# abstract = driver.find_elements(
#     By.CSS_SELECTOR, 'blockquote.tight'
# )[2].text
# abstract

## Integrating

In [198]:
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [ ]:
urlBase = 'https://convention2.allacademic.com/one/ica/ica'
# scrape 2005~2013
years = range(5,14)
for year in years:
    year = str(year).zfill(2)
    url = urlBase + year
    driver.get(url)
    click_on_view_program()
    click_on_individual_presentations()
    paper_meta_dict_list = []
    author_dict_list = []
    iterators = get_iterators()
    total_pages = int(iterators[-2].text)
    for i in range(1,total_pages+1):
        page_num = i
#         print(f'going to solve page {i}')
        if i < 10:
            pass
        elif i >= 10 and i < 17:
            select = Select(driver.find_element(
                By.XPATH, '//child::div[@class="iterator"][1] // select'
            ))
            select.select_by_visible_text('+ 10')
        elif i >= 17 and i < 27:
            select = Select(driver.find_element(
                By.XPATH, '//child::div[@class="iterator"][1] // select'
            ))
            select.select_by_visible_text('+ 20')
        else:
            iterators = get_iterators()
            iterators[-2].click()
        iterators = get_iterators()
        for j in iterators:
            if (j.text == str(i)):
                current_idx = int(j.text)
                j.click()
                break 
        scrape_one_page(f'20{year}',page_num)
        print(f'page {i} is done')
        iterators = get_iterators()
        iterators[1].click()
    print(f'20{year} is done!')
    paper_df = pd.DataFrame(paper_meta_dict_list)
    author_df = pd.DataFrame(author_dict_list)
    paper_df.to_csv(f'../data/interim/paper_df_20{year}.csv', index = False)
    author_df.to_csv(f'../data/interim/author_df_20{year}.csv', index = False)
driver.close()
driver.quit()

Page 1 Paper 1 is done
Page 1 Paper 2 is done
Page 1 Paper 3 is done
Page 1 Paper 4 is done
Page 1 Paper 5 is done
Page 1 Paper 6 is done
Page 1 Paper 7 is done
Page 1 Paper 8 is done
Page 1 Paper 9 is done
Page 1 Paper 10 is done
Page 1 Paper 11 is done
Page 1 Paper 12 is done
Page 1 Paper 13 is done
Page 1 Paper 14 is done
Page 1 Paper 15 is done
Page 1 Paper 16 is done
Page 1 Paper 17 is done
Page 1 Paper 18 is done
page 1 is done
Page 2 Paper 1 is done
Page 2 Paper 2 is done
Page 2 Paper 3 is done
Page 2 Paper 4 is done
Page 2 Paper 5 is done
Page 2 Paper 6 is done
Page 2 Paper 7 is done
Page 2 Paper 8 is done
Page 2 Paper 9 is done
Page 2 Paper 10 is done
Page 2 Paper 11 is done
Page 2 Paper 12 is done
Page 2 Paper 13 is done
Page 2 Paper 14 is done
Page 2 Paper 15 is done
Page 2 Paper 16 is done
Page 2 Paper 17 is done
Page 2 Paper 18 is done
Page 2 Paper 19 is done
Page 2 Paper 20 is done
Page 2 Paper 21 is done
Page 2 Paper 22 is done
Page 2 Paper 23 is done
Page 2 Paper 24 is 

Page 11 Paper 15 is done
Page 11 Paper 16 is done
Page 11 Paper 17 is done
Page 11 Paper 18 is done
Page 11 Paper 19 is done
Page 11 Paper 20 is done
Page 11 Paper 21 is done
Page 11 Paper 22 is done
Page 11 Paper 23 is done
Page 11 Paper 24 is done
Page 11 Paper 25 is done
Page 11 Paper 26 is done
Page 11 Paper 27 is done
Page 11 Paper 28 is done
Page 11 Paper 29 is done
Page 11 Paper 30 is done
Page 11 Paper 31 is done
Page 11 Paper 32 is done
Page 11 Paper 33 is done
page 11 is done
Page 12 Paper 1 is done
Page 12 Paper 2 is done
Page 12 Paper 3 is done
Page 12 Paper 4 is done
Page 12 Paper 5 is done
Page 12 Paper 6 is done
Page 12 Paper 7 is done
Page 12 Paper 8 is done
Page 12 Paper 9 is done
Page 12 Paper 10 is done
Page 12 Paper 11 is done
Page 12 Paper 12 is done
Page 12 Paper 13 is done
Page 12 Paper 14 is done
Page 12 Paper 15 is done
Page 12 Paper 16 is done
Page 12 Paper 17 is done
Page 12 Paper 18 is done
Page 12 Paper 19 is done
Page 12 Paper 20 is done
Page 12 Paper 21 i

Page 21 Paper 21 is done
Page 21 Paper 22 is done
Page 21 Paper 23 is done
Page 21 Paper 24 is done
Page 21 Paper 25 is done
Page 21 Paper 26 is done
Page 21 Paper 27 is done
Page 21 Paper 28 is done
page 21 is done
Page 22 Paper 1 is done
Page 22 Paper 2 is done
Page 22 Paper 3 is done
Page 22 Paper 4 is done
Page 22 Paper 5 is done
Page 22 Paper 6 is done
Page 22 Paper 7 is done
Page 22 Paper 8 is done
Page 22 Paper 9 is done
Page 22 Paper 10 is done
Page 22 Paper 11 is done
Page 22 Paper 12 is done
Page 22 Paper 13 is done
Page 22 Paper 14 is done
Page 22 Paper 15 is done
Page 22 Paper 16 is done
Page 22 Paper 17 is done
Page 22 Paper 18 is done
Page 22 Paper 19 is done
Page 22 Paper 20 is done
Page 22 Paper 21 is done
Page 22 Paper 22 is done
Page 22 Paper 23 is done
Page 22 Paper 24 is done
Page 22 Paper 25 is done
Page 22 Paper 26 is done
Page 22 Paper 27 is done
Page 22 Paper 28 is done
Page 22 Paper 29 is done
Page 22 Paper 30 is done
page 22 is done
Page 23 Paper 1 is done
Pag

Page 32 Paper 6 is done
Page 32 Paper 7 is done
Page 32 Paper 8 is done
Page 32 Paper 9 is done
Page 32 Paper 10 is done
Page 32 Paper 11 is done
Page 32 Paper 12 is done
Page 32 Paper 13 is done
Page 32 Paper 14 is done
Page 32 Paper 15 is done
Page 32 Paper 16 is done
Page 32 Paper 17 is done
Page 32 Paper 18 is done
Page 32 Paper 19 is done
Page 32 Paper 20 is done
Page 32 Paper 21 is done
Page 32 Paper 22 is done
Page 32 Paper 23 is done
Page 32 Paper 24 is done
Page 32 Paper 25 is done
Page 32 Paper 26 is done
Page 32 Paper 27 is done
Page 32 Paper 28 is done
Page 32 Paper 29 is done
Page 32 Paper 30 is done
Page 32 Paper 31 is done
Page 32 Paper 32 is done
Page 32 Paper 33 is done
page 32 is done
Page 33 Paper 1 is done
Page 33 Paper 2 is done
Page 33 Paper 3 is done
Page 33 Paper 4 is done
Page 33 Paper 5 is done
Page 33 Paper 6 is done
Page 33 Paper 7 is done
Page 33 Paper 8 is done
Page 33 Paper 9 is done
Page 33 Paper 10 is done
Page 33 Paper 11 is done
Page 33 Paper 12 is do

Page 3 Paper 21 is done
Page 3 Paper 22 is done
Page 3 Paper 23 is done
Page 3 Paper 24 is done
Page 3 Paper 25 is done
Page 3 Paper 26 is done
Page 3 Paper 27 is done
Page 3 Paper 28 is done
Page 3 Paper 29 is done
Page 3 Paper 30 is done
Page 3 Paper 31 is done
Page 3 Paper 32 is done
Page 3 Paper 33 is done
page 3 is done
Page 4 Paper 1 is done
Page 4 Paper 2 is done
Page 4 Paper 3 is done
Page 4 Paper 4 is done
Page 4 Paper 5 is done
Page 4 Paper 6 is done
Page 4 Paper 7 is done
Page 4 Paper 8 is done
Page 4 Paper 9 is done
Page 4 Paper 10 is done
Page 4 Paper 11 is done
Page 4 Paper 12 is done
Page 4 Paper 13 is done
Page 4 Paper 14 is done
Page 4 Paper 15 is done
Page 4 Paper 16 is done
Page 4 Paper 17 is done
Page 4 Paper 18 is done
Page 4 Paper 19 is done
Page 4 Paper 20 is done
Page 4 Paper 21 is done
Page 4 Paper 22 is done
Page 4 Paper 23 is done
Page 4 Paper 24 is done
Page 4 Paper 25 is done
Page 4 Paper 26 is done
Page 4 Paper 27 is done
Page 4 Paper 28 is done
Page 4 Pap

...

Page 26 Paper 43 is done
Page 26 Paper 44 is done
Page 26 Paper 45 is done
Page 26 Paper 46 is done
Page 26 Paper 47 is done
Page 26 Paper 48 is done
Page 26 Paper 49 is done
Page 26 Paper 50 is done
page 26 is done
Page 27 Paper 1 is done
Page 27 Paper 2 is done
Page 27 Paper 3 is done
Page 27 Paper 4 is done
Page 27 Paper 5 is done
Page 27 Paper 6 is done
Page 27 Paper 7 is done
Page 27 Paper 8 is done
Page 27 Paper 9 is done
Page 27 Paper 10 is done
Page 27 Paper 11 is done
Page 27 Paper 12 is done
Page 27 Paper 13 is done
Page 27 Paper 14 is done
Page 27 Paper 15 is done
Page 27 Paper 16 is done
Page 27 Paper 17 is done
Page 27 Paper 18 is done
Page 27 Paper 19 is done
Page 27 Paper 20 is done
Page 27 Paper 21 is done
Page 27 Paper 22 is done
Page 27 Paper 23 is done
Page 27 Paper 24 is done
Page 27 Paper 25 is done
Page 27 Paper 26 is done
Page 27 Paper 27 is done
Page 27 Paper 28 is done
Page 27 Paper 29 is done
Page 27 Paper 30 is done
Page 27 Paper 31 is done
Page 27 Paper 32 i

Page 33 Paper 19 is done
Page 33 Paper 20 is done
Page 33 Paper 21 is done
Page 33 Paper 22 is done
Page 33 Paper 23 is done
Page 33 Paper 24 is done
Page 33 Paper 25 is done
Page 33 Paper 26 is done
Page 33 Paper 27 is done
Page 33 Paper 28 is done
Page 33 Paper 29 is done
Page 33 Paper 30 is done
Page 33 Paper 31 is done
Page 33 Paper 32 is done
Page 33 Paper 33 is done
Page 33 Paper 34 is done
Page 33 Paper 35 is done
Page 33 Paper 36 is done
Page 33 Paper 37 is done
Page 33 Paper 38 is done
Page 33 Paper 39 is done
Page 33 Paper 40 is done
Page 33 Paper 41 is done
Page 33 Paper 42 is done
Page 33 Paper 43 is done
Page 33 Paper 44 is done
Page 33 Paper 45 is done
Page 33 Paper 46 is done
Page 33 Paper 47 is done
Page 33 Paper 48 is done
Page 33 Paper 49 is done
Page 33 Paper 50 is done
page 33 is done
Page 34 Paper 1 is done
Page 34 Paper 2 is done
Page 34 Paper 3 is done
Page 34 Paper 4 is done
Page 34 Paper 5 is done
Page 34 Paper 6 is done
Page 34 Paper 7 is done
Page 34 Paper 8 

Page 1 Paper 28 is done
Page 1 Paper 29 is done
Page 1 Paper 30 is done
Page 1 Paper 31 is done
Page 1 Paper 32 is done
Page 1 Paper 33 is done
Page 1 Paper 34 is done
Page 1 Paper 35 is done
Page 1 Paper 36 is done
Page 1 Paper 37 is done
Page 1 Paper 38 is done
Page 1 Paper 39 is done
Page 1 Paper 40 is done
Page 1 Paper 41 is done
Page 1 Paper 42 is done
Page 1 Paper 43 is done
Page 1 Paper 44 is done
Page 1 Paper 45 is done
Page 1 Paper 46 is done
Page 1 Paper 47 is done
Page 1 Paper 48 is done
Page 1 Paper 49 is done
Page 1 Paper 50 is done
page 1 is done
Page 2 Paper 1 is done
Page 2 Paper 2 is done
Page 2 Paper 3 is done
Page 2 Paper 4 is done
Page 2 Paper 5 is done
Page 2 Paper 6 is done
Page 2 Paper 7 is done
Page 2 Paper 8 is done
Page 2 Paper 9 is done
Page 2 Paper 10 is done
Page 2 Paper 11 is done
Page 2 Paper 12 is done
Page 2 Paper 13 is done
Page 2 Paper 14 is done
Page 2 Paper 15 is done
Page 2 Paper 16 is done
Page 2 Paper 17 is done
Page 2 Paper 18 is done
Page 2 Pap

Page 8 Paper 18 is done
Page 8 Paper 19 is done
Page 8 Paper 20 is done
Page 8 Paper 21 is done
Page 8 Paper 22 is done
Page 8 Paper 23 is done
Page 8 Paper 24 is done
Page 8 Paper 25 is done
Page 8 Paper 26 is done
Page 8 Paper 27 is done
Page 8 Paper 28 is done
Page 8 Paper 29 is done
Page 8 Paper 30 is done
Page 8 Paper 31 is done
Page 8 Paper 32 is done
Page 8 Paper 33 is done
Page 8 Paper 34 is done
Page 8 Paper 35 is done
Page 8 Paper 36 is done
Page 8 Paper 37 is done
Page 8 Paper 38 is done
Page 8 Paper 39 is done
Page 8 Paper 40 is done
Page 8 Paper 41 is done
Page 8 Paper 42 is done
Page 8 Paper 43 is done
Page 8 Paper 44 is done
Page 8 Paper 45 is done
Page 8 Paper 46 is done
Page 8 Paper 47 is done
Page 8 Paper 48 is done
Page 8 Paper 49 is done
Page 8 Paper 50 is done
page 8 is done
Page 9 Paper 1 is done
Page 9 Paper 2 is done
Page 9 Paper 3 is done
Page 9 Paper 4 is done
Page 9 Paper 5 is done
Page 9 Paper 6 is done
Page 9 Paper 7 is done
Page 9 Paper 8 is done
Page 9 Pa

Page 14 Paper 48 is done
Page 14 Paper 49 is done
Page 14 Paper 50 is done
page 14 is done
Page 15 Paper 1 is done
Page 15 Paper 2 is done
Page 15 Paper 3 is done
Page 15 Paper 4 is done
Page 15 Paper 5 is done
Page 15 Paper 6 is done
Page 15 Paper 7 is done
Page 15 Paper 8 is done
Page 15 Paper 9 is done
Page 15 Paper 10 is done
Page 15 Paper 11 is done
Page 15 Paper 12 is done
Page 15 Paper 13 is done
